In [ ]:
!pip install numpy_financial
!pip install yfinance

In [ ]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import random

In [ ]:
def get_average__monthly_volume(stock_volume):
    # # get the ticker
    # ticker = yf.Ticker(ticker_symbol)
    # # get the stock data
    # stock_data = ticker.history(start='2023-01-01', end='2023-10-31')

    # # Resample data to monthly frequency and calculate average volume
    # stock_volume = stock_data['Volume']
    # Resample data to monthly frequency and calculate the number of trading days per month
    trading_days_per_month = stock_volume.resample('M').count()

    # Filter out months with less than 18 trading days
    valid_months = trading_days_per_month[trading_days_per_month >= 18]

    # Filter the stock data for the valid months
    stock_volume = stock_volume[stock_volume.index.month.isin(valid_months.index.month)]

    # return the result
    return stock_volume.sum()/len(valid_months)

In [ ]:
def validtickers(df):
    ticker_lst = []
    ##start and end dates to check if the ticker is avaliable in those times
    start_date = '2023-01-01'
    end_date = '2023-10-01'
    ##in case there's no title and the column title is a ticker (so in the ticker_example.csv, the title was AAPL, which is a ticker)
    try:
        ticker = df.columns[0]
        stock = yf.Ticker(ticker)
        ##get historical data from it
        history = stock.history(start=start_date, end=end_date)
        ##if it is a valid ticker, this would run and if it hits the requirements, it appends the ticker to the tickerlist
        if ((stock.fast_info['currency'] == "CAD" or stock.fast_info['currency'] == "USD") and (get_average__monthly_volume(history['Volume']) > 150000)): # added my function here
          ticker_lst.append(df.columns[0])
          x = 0
        else:
          ##otherwise, it's not a ticker that hits the requirements,
          print('not a valid stock')
          x = 0
    except:
      ##if the code outputs an error, the ticker isn't valid, so outputs not a ticker
        print('not a ticker')
        x = 0
    while x < len(df):
      ##for the rest of the column, exactly the same process as above, try getting an output from the ticker, and if it doesn't work, go next
        try:
            ticker = df.iloc[x,0]
            stock = yf.Ticker(ticker)
            history = stock.history(start=start_date, end=end_date)
            if ((stock.fast_info['currency'] == 'CAD' or stock.fast_info['currency'] == 'USD') and (get_average__monthly_volume(history['Volume']) > 150000)):
             ticker_lst.append(df.iloc[x,0])
             x += 1
            else:
              print('not a valid stock')
              x += 1
        except:
            print('not a ticker')
            x += 1

    return ticker_lst
    ##return random.sample(ticker_lst, random.randint(10,22))

tickerlist = validtickers(pd.read_csv('Tickers_Example.csv'))
print(tickerlist)

ERROR:yfinance:AGN: No timezone found, symbol may be delisted


not a ticker


ERROR:yfinance:CELG: No timezone found, symbol may be delisted


not a ticker


ERROR:yfinance:MON: No timezone found, symbol may be delisted


not a ticker


ERROR:yfinance:RTN: No timezone found, symbol may be delisted


not a ticker
['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


In [ ]:
stock_currency = pd.DataFrame({'Ticker':[],
                         'Currency':[]})
for i in tickerlist:
  tempdf = pd.DataFrame(columns=['Ticker','Currency'])
  temp = yf.Ticker(i)
  tempdf['Ticker'] = [i]
  ##Code for industry if avaliable
  tempdf['Currency'] = [temp.fast_info['currency']]
  stock_currency = pd.concat([stock_currency, tempdf],ignore_index=True)

stock_currency

,Ticker,Currency
0,AAPL,USD
1,ABBV,USD
2,ABT,USD
3,ACN,USD
4,AIG,USD
5,AMZN,USD
6,AXP,USD
7,BA,USD
8,BAC,USD
9,BIIB,USD


In [ ]:
pctchangelist = pd.DataFrame()
stockprice = pd.DataFrame()

# Define the date range
start_date = '2023-01-01'
end_date = '2023-10-01'

# Loop through each ticker
for ticker_symbol in tickerlist:
    # Retrieve the historical data for the ticker
    ticker = yf.Ticker(ticker_symbol)

    #Make a list with the stock values
    stockprice[ticker_symbol] = ticker.history(start=start_date, end=end_date).Close

    # Extract and calculate the percentage change of closing prices
    pct_change = ticker.history(start=start_date, end=end_date).Close.pct_change()

    # Add the percentage change data to the DataFrame
    pctchangelist[ticker_symbol] = pct_change

# Drop the first row (index 0) since it contains NaN values due to percentage change calculation
pctchangelist = pctchangelist.dropna()
pctchangelist2 = pctchangelist

# Localize timezone to only display date
pctchangelist.index = pctchangelist.index.tz_localize(None)
stockprice.index = stockprice.index.tz_localize(None)

# Display the resulting DataFrame
stockprice

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,124.374802,156.074890,107.430855,266.033325,61.738464,85.820000,144.975815,195.389999,32.757507,272.630005,...,105.070328,122.739281,48.790001,25.214357,84.689217,158.244385,512.690674,203.609467,168.576355,42.753727
2023-01-04,125.657639,157.334015,109.028885,265.127686,62.650860,85.139999,148.345978,203.639999,33.373360,270.809998,...,109.314308,123.832169,50.610001,25.520916,85.790459,164.023041,498.712860,205.237717,170.336365,44.084988
2023-01-05,124.325081,157.141800,108.626930,258.867188,62.307484,83.120003,144.796021,204.990005,33.304935,271.589996,...,107.226631,123.467857,48.830002,25.492176,84.384216,161.851181,484.339630,199.195541,167.191406,43.740200
2023-01-06,128.899506,160.082977,110.126923,264.999725,63.327801,86.080002,148.494308,213.000000,33.637299,279.250000,...,113.048622,125.040108,49.560001,25.884954,84.345154,169.830795,484.379181,207.954727,172.105972,44.353161
2023-01-09,129.426544,155.382843,109.950455,269.468750,62.660671,87.360001,148.721741,208.570007,33.128975,274.720001,...,112.333122,125.970016,49.810001,25.827475,84.081490,171.304550,484.438507,207.415253,174.741180,44.640484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,175.848328,153.117371,96.933411,315.680695,62.130001,131.270004,152.602997,198.779999,27.600000,255.020004,...,110.430000,118.947205,72.110001,22.719999,82.419998,159.438492,510.279999,208.479996,152.523773,33.094208
2023-09-26,171.733749,152.384705,95.700241,309.327057,61.259998,125.980003,150.501617,195.639999,27.170000,258.049988,...,109.720001,117.406288,69.639999,22.430000,81.379997,156.525208,505.450012,204.839996,150.141663,32.532619
2023-09-27,170.205750,151.612442,95.004089,313.081512,61.009998,125.980003,149.286621,195.460007,27.270000,257.519989,...,109.190002,115.865372,70.040001,22.070000,81.230003,156.703568,503.730011,202.399994,150.645767,32.040001


In [ ]:
# optional that we can add to reduce the number of tickers by removing the most volatile stocks
# might cause problems if there are only like 45 stocks and the remaining ones are from similar industries -> hence the lower volatility
# given that it's being run for only like two weeks, doubt it matters too much
if len(tickerlist) > 44:
  # create a list of averages, append the average of the absolute value to get the variation from zero
  # since its percent change, higher average percent change means high volatility
    avelist = []
    for i in tickerlist:
        avelist.append(pctchangelist[i].abs().mean())

    avelist = pd.Series(avelist)
    # get the median, cut the number of stocks in half
    avemid = stdlist.median()
    i = 0
    while i < len(stdlist):
      # remove them from the percent change list if the stock average is above the overall average
        if avelist[i] > avemid:
            print(avelist[i])
            pctchangelist = pctchangelist.drop(columns = [tickerlist[i]])
            i += 1
        else:
            i += 1
pctchangelist

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-04,0.010314,0.008067,0.014875,-0.003404,0.014778,-0.007924,0.023246,0.042223,0.018800,-0.006676,...,0.040392,0.008904,0.037303,0.012158,0.013003,0.036517,-0.027264,0.007997,0.010440,0.031138
2023-01-05,-0.010605,-0.001222,-0.003687,-0.023613,-0.005481,-0.023726,-0.023930,0.006629,-0.002050,0.002880,...,-0.019098,-0.002942,-0.035171,-0.001126,-0.016392,-0.013241,-0.028821,-0.029440,-0.018463,-0.007821
2023-01-06,0.036794,0.018717,0.013809,0.023690,0.016376,0.035611,0.025541,0.039075,0.009979,0.028204,...,0.054296,0.012734,0.014950,0.015408,-0.000463,0.049302,0.000082,0.043973,0.029395,0.014014
2023-01-09,0.004089,-0.029361,-0.001602,0.016864,-0.010535,0.014870,0.001532,-0.020798,-0.015112,-0.016222,...,-0.006329,0.007437,0.005044,-0.002221,-0.003126,0.008678,0.000122,-0.002594,0.015312,0.006478
2023-01-10,0.004457,-0.012495,0.015158,0.004310,-0.006576,0.028732,0.010638,-0.009014,0.006787,0.024534,...,0.020068,0.011416,-0.016061,0.001113,-0.001161,0.009735,-0.008285,0.003878,-0.016071,0.001073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,0.007380,0.012505,-0.006928,0.002847,0.008931,0.016651,0.000980,0.005412,-0.001447,-0.010323,...,0.025539,0.001914,0.007545,-0.007860,0.008072,0.003680,0.008259,-0.003632,0.003446,0.006291
2023-09-26,-0.023398,-0.004785,-0.012722,-0.020127,-0.014003,-0.040299,-0.013770,-0.015796,-0.015580,0.011881,...,-0.006429,-0.012955,-0.034253,-0.012764,-0.012618,-0.018272,-0.009465,-0.017460,-0.015618,-0.016969
2023-09-27,-0.008897,-0.005068,-0.007274,0.012137,-0.004081,0.000000,-0.008073,-0.000920,0.003681,-0.002054,...,-0.004830,-0.013125,0.005744,-0.016050,-0.001843,0.001139,-0.003403,-0.011912,0.003358,-0.015142


In [ ]:
tempchangelist = pctchangelist
# get all negative correlations
# to be a safe portfolio, we want stocks that cancel each other out, so any big change in one stock is slightly negated by the other stock
# thus, we want the lowest correlation values, at least below zero
# this guarantees at least two stocks, even if all correlation values are positive
corrvalues = tempchangelist .corr()
# get the minimum of the correlation values
minvalues = corrvalues.min()
# correlation dataframe is sorted in a way such that the smallest value occurs twice, once in both stocks
stocks = np.where(minvalues == minvalues.min())[0]
# get the index and find the stock associated with the index - there would be two
firststock = tickerlist[stocks[0]]
secondstock = tickerlist[stocks[1]]
# create a portfolio pct change and remove the two stocks from the percent change dataframe
# then we can compare them percent change dataframe again to get new correlation values
stockprice['Portfolio'] = stockprice[firststock] + stockprice[secondstock]
tempchangelist['Portfolio'] = stockprice['Portfolio'].ffill().pct_change().dropna()
tempchangelist = tempchangelist.drop(columns=[firststock, secondstock])
# create a list with the stocks and a list with the correlation values
# since correlation can't be above 1, set the starting index to 2
stocklist = [[firststock, secondstock]]

i = 2
while i < 22:
  # loop at max 22 times for 22 stocks
  # create a temporary list for the correlation values and get the minimum for the last column (portfolio column)
    temp = tempchangelist.corr()
    mintemp = temp.iloc[:, -1].min()
    # if the minimum is bigger than 0, then there is a positive correlation, no matter how slight, so we woul rather not include it
    if mintemp > 0:
      # then get the minimum value in the list, which should be lower than if the correlation is positive
        minvalues = temp.min()
        # if it is still positive, then there is only positive correlations between the stocks left, so break and we fill the rest of the
        # stocks in the following block of code - this guarantees the minimum isn't between the portfolio and one of the stocks
        if minvalues.min() > 0:
            break
        else:
          # if the minimum isn't above zero, we find which stocks are associated with the low correlation, and add them to the list
          # same process as above
            stocks = np.where(minvalues == minvalues.min())[0]
            firststock = tempchangelist.columns[stocks[0]]
            secondstock = tempchangelist.columns[stocks[1]]
            stockprice['Portfolio'] = stockprice[firststock] + stockprice[secondstock]
            tempchangelist['Portfolio'] = stockprice['Portfolio'].ffill().pct_change().dropna()
            tempchangelist = tempchangelist.drop(columns=[firststock, secondstock])
            stocklist = stocklist + [[firststock, secondstock]]
            # add two because adding two new stocks
            i += 2
    else:
      # else, there is a negative correlation between the portfolio and one of the stocks, in which case
      # find the index associated with the number and add it to the portfolio and the stock/correlation list, and remove it from
      # the pctchange list
        minvalues = mintemp
        stock = temp.iloc[:, -1].idxmin()
        stockprice['Portfolio'] = stockprice['Portfolio'] + stockprice[stock]
        tempchangelist['Portfolio'] = stockprice['Portfolio'].ffill().pct_change().dropna()
        tempchangelist = tempchangelist.drop(columns=[stock])
        stocklist[len(stocklist)-1].append(stock)
        i += 1

print(stocklist)

# flatten the stocklist, as to find the number of stocks theyre are
def flatten(A):
    lst = []
    for i in A:
        if isinstance(i,list): lst.extend(flatten(i))
        else: lst.append(i)
    return lst

flatstocklist = flatten(stocklist)
print(flatstocklist)

[['SHOP.TO', 'UNH'], ['C', 'LLY', 'BAC'], ['ABBV', 'AMZN'], ['ACN', 'MRK'], ['AXP', 'CL'], ['BMY', 'QCOM']]
['SHOP.TO', 'UNH', 'C', 'LLY', 'BAC', 'ABBV', 'AMZN', 'ACN', 'MRK', 'AXP', 'CL', 'BMY', 'QCOM']


In [ ]:
# make new correlation dataframe with the new percent change list
corrvalues2 = pctchangelist2.corr()
# remove all items in the stocklist from the rows and all items not in the stocklist from the columns
# this guarantees 22 different stocks, which is safer for our portfolio, and will be weighted according to the correlation
corrvalues2 = corrvalues2.drop(columns = [x for x in flatstocklist])
corrvalues2 = corrvalues2.drop(columns = ['Portfolio'])
corrvalues2 = corrvalues2.drop([x for x in tickerlist if x not in flatstocklist])

# get the remining stocks by finding the highest (22 - however many current stocks) number of correlation values
maxcorr = pd.DataFrame(corrvalues2.stack().sort_values(ascending = False).iloc[:24])
maxcorr = maxcorr.tail(22-len(stocklist))

# we love recursion
# go through the entire list and find where the index equals the stock associated with the correlation values
# outputs in a way such that if there's correlation, list becomes nested - eg.
# [['SHOP.TO', 'UNH'], [['C', 'BLK', 'C'], 'LLY', ['BAC', 'TD.TO', 'BAC']], ['ABBV', 'AMZN'], [['ACN', 'TXN', 'ACN'], 'MRK'],
# [['AXP', 'BK', 'AXP'], ['CL', 'KO', 'CL']], [['BMY', 'PFE'], ['QCOM', 'BLK']]]

# go through the list of correlations
i = 0
while i < len(maxcorr):
  # go through the list of stocks to find the appropriate one
    j = 0
    while j < len(stocklist):
      # since each is a nested list, go through each of the nested lists
        k = 0
        while k < len(stocklist[j]):
          # if the first is already a stock, we want it shared amongst at most three, so we check for that
            if isinstance(stocklist[j][k], list):
                if maxcorr.index[i][0] == stocklist[j][k][0]:
                    if len(stocklist[j][k]) < 3:
                      # if it's less than three, append the stock, else break
                        stocklist[j][k].append(maxcorr.index[i][0])
                        break
                    else:
                        break
                else:
                  # if it's not the first item, keep checking the nested list
                    k += 1
            elif (maxcorr.index[i][0] == stocklist[j][k]):
              # if it's not a list, check it's index, then if it matches, create a list and break
                stocklist[j][k] = [maxcorr.index[i][0], maxcorr.index[i][1]]
                break
                # otherwise, check the next one
            else: k += 1
        j += 1
    i += 1


print(stocklist)

[['SHOP.TO', 'UNH'], [['C', 'BLK', 'C'], 'LLY', ['BAC', 'TD.TO', 'BAC']], ['ABBV', 'AMZN'], [['ACN', 'TXN', 'ACN'], 'MRK'], [['AXP', 'BK', 'AXP'], ['CL', 'KO', 'CL']], [['BMY', 'PFE'], ['QCOM', 'BLK']]]


In [ ]:
# averageBeta returns the average beta of list of stocks, or the beta of a single stock
def averageBeta (element):
    # element ((listof Str) or Str): Either a list of stocks to be recursively called on, or a stock,
    #                                represented by its ticker as a string
    if type(element) is list:
        count = 0
        for i in range (len(element)):
            count += averageBeta(element[i])
        return count/(len(element))
    return yf.Ticker(element).info['beta']

# splitPercentages divides the % of the portfolio evenly amongst all stocks in tickers, and adds a bonus
#                  percent if the ticker is
def splitPercentages (tickers, total, added):
    # element ((listof Str) or Str): Either a list of stocks to be recursively called on, or a stock,
    #                                represented by its ticker as a string
    # total (0 <= Num <= 1): The percentage to be divided amongst the stocks
    allTickers = []
    allWeights = []
    for i in range (len(tickers)):
        if type(tickers[i]) is list:
            if i > (len(tickers)/2 - 1):
                added = 0
            nested = splitPercentages(tickers[i], total/len(tickers), added/len(tickers[i]))
            allTickers = allTickers + nested[0]
            allWeights = allWeights + nested[1]
            continue
        if i > (len(tickers)/2 - 1):
            added = 0
        allTickers.append(tickers[i])
        allWeights.append(total/len(tickers) + added*2)
    return [allTickers, allWeights]

# assignWeights returns a list of length 2. The first element in the list is a list of stocks, and the second
#               element is a list, where the element at index i is the weight of the stock at index i in the
#               first list
def assignWeights (tickers):
    # tickers (listof (Str or (listof Str))):

    temp = tickers
    # sorting the groups by their beta, or their average beta
    temp.sort(key=averageBeta)

    # we remove a portion from the total percentage to be set aside for biased redistribution later
    totalPercentage = 1-(len(temp)*0.005)
    returns = splitPercentages(temp, totalPercentage, 0.005)

    # this occurs when len(temp) is odd. However, it is a happy accident, as now we can add more weight to the
    # stock with the lowest beta
    if sum(returns[1]) < 1:
        returns[1][0] += 1-sum(returns[1])

    return returns

stock_weight = assignWeights(stocklist)
print(stock_weight)

HTTPError: ignored

In [ ]:
def price(ticker):
    start_date = '2023-10-23'
    end_date = '2023-10-24'
    stock = yf.Ticker(ticker)
    stock_hist = stock.history(start=start_date, end=end_date)
    return stock_hist.Close.iloc[0]
def get_num_shares(price, value):
    return value / price

def final_data_clean(stocks, weighing):
    total_value = 750000
    final = pd.DataFrame({"Ticker": stocks}, index=range(1, len(stocks) + 1))
    initial_prices = []
    currencies = []
    values = []
    shares = []
    for i in range(len(stocks)):
        curr_price = price(stocks[i])
        initial_prices.append(curr_price)
        currencies.append(stock_currency[stock_currency['Ticker'] == stocks[i]].iloc[0, 1])
        curr_value = total_value * weighing[i]
        values.append(curr_value)
        shares.append(get_num_shares(curr_price, curr_value))
    final['Price'] = initial_prices
    final['Currency'] = currencies
    final['Shares'] = shares
    final['Value'] = values
    final['Weight'] = weighing
    return final
Portfolio_Final = final_data_clean(flatten(stocklist), [0.25, 0.25, 0.25, 0.25])
print(Portfolio_Final)
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv('Stocks_Group_10.csv')

  Ticker       Price Currency       Shares     Value  Weight
1   AAPL  172.772385      USD  1085.242878  187500.0    0.25
2   ABBV  144.729996      USD  1295.515826  187500.0    0.25
3    ABT   95.779999      USD  1957.611217  187500.0    0.25
4     MO   42.130001      USD  4450.510212  187500.0    0.25


In [ ]:
# define function to get the price of a ticker
def price(ticker):
    # set start and end date
    start_date = '2023-10-23'
    end_date = '2023-10-24'
    stock = yf.Ticker(ticker)
    # get the ticker history
    stock_hist = stock.history(start=start_date, end=end_date)
    # return the first cell of the data
    return stock_hist.Close.iloc[0]
# create a function to get the currency exchange rate from that currency to CAD
def usd_exchange():
    exchange_ticker = yf.Ticker('CADUSD=x')
    # get data history
    start_date = '2023-10-23'
    end_date = '2023-10-24'
    exchange_hist = exchange_ticker.history(start=start_date, end=end_date)
    # make a dataframe from the closing exchange rate
    exchange_rates = pd.DataFrame(1/exchange_hist.Close)
    exchange_rates.columns = ['cad']
    # return the first cell of the exchange rate
    return exchange_rates.iloc[0,0]
# get the current usd exchange rate
usd_exchange_rate = usd_exchange()
# function to get number of shares of a stock given the price and value
def get_num_shares(price, value, currency):
    # if currency is usd we consider the usd exchange rate
    if currency == 'USD':
        return value / (price * usd_exchange_rate)
    else:
        return value / price

# function to get the cleaned version of the output
def final_data_clean(stocks, weighing):
    # set initial total value
    total_value = 750000
    # create a final dataframe with a ticker column
    final = pd.DataFrame({"Ticker": stocks}, index=range(1, len(stocks) + 1))
    # set an initial empty list to hold every initial price, currency, value, and shares for each stock
    initial_prices = []
    currencies = []
    values = []
    shares = []
    # loop through each stock to get the initial price, currency, value, and shares of every stock
    for i in range(len(stocks)):
        curr_price = price(stocks[i])
        initial_prices.append(curr_price)
        curr_currency = stocklist[stocklist['Ticker'] == stocks[i]].iloc[0, 1]
        currencies.append(curr_currency)
        curr_value = total_value * weighing[i]
        values.append(curr_value)
        shares.append(get_num_shares(curr_price, curr_value, curr_currency))
    # create a column for each set of data
    final['Price'] = initial_prices
    final['Currency'] = currencies
    final['Shares'] = shares
    final['Value'] = values
    final['Weight'] = weighing
    # return the final dataframe
    return final
Portfolio_Final = final_data_clean(['AAPL', 'ABBV', 'ABT', 'MO'], [0.25, 0.25, 0.25, 0.25])
print(Portfolio_Final)
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv('Stocks_Group_10.csv')